<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/EconomicDispatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Economic Dispatch 

This Jupyter Notebook solves an instance of the Economic Dispatch Problem described in XXX

## Requirements

In [12]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
glpk = pe.SolverFactory('glpk', executable='/usr/bin/glpsol')
import pandas as pd
import random
import pyomo.environ as pe

## Input Data

In [13]:
gen = pd.DataFrame({
       'unit': [1,   2], 
       'cost': [10,  20], 
       'pmin': [0,   0], 
       'pmax': [100, 100]})
dem = 100

## Solving the Economic Dispatch

In [17]:
# Model
m = pe.ConcreteModel()
# Sets
m.g = pe.Set(initialize=list(range(len(gen))))
# Variables
m.p = pe.Var(m.g,within=pe.NonNegativeReals)
def obj_rule(m):
  return sum(gen.loc[g,'cost']*m.p[g] for g in m.g)
m.obj = pe.Objective(rule=obj_rule)
# Energy balance
def bal_rule(m):
  return sum(m.p[g] for g in m.g) == dem
m.bal = pe.Constraint(rule=bal_rule)
# Maximum generation
def max_gen_rule(m,g):
  return m.p[g] <= gen.loc[g,'pmax']
m.max_gen = pe.Constraint(m.g, rule=max_gen_rule)
# We solve the optimization problem using GLPK
glpk.solve(m).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1000.0
  Upper bound: 1000.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 5
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.011249780654907227
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

## Solution Output

In [15]:
# Print results
print('obj =',m.obj())
print('p1 =',m.p[0].value)
print('p2 =',m.p[1].value)

obj = 1000.0
p1 = 100.0
p2 = 0.0
